In [1]:
import numpy as np
import astropy.io.fits as fits
import pandas as pd
import math
pd.set_option('display.max_colwidth', None)

from alerce.core import Alerce
alerce = Alerce()

# Useful
# http://ps1images.stsci.edu/ps1_dr2_api.html

from astropy.io import ascii
from astropy.table import Table

import sys
import re
import numpy as np
import matplotlib.pyplot as plt
import json
import requests

try: # Python 3.x
    from urllib.parse import quote as urlencode
    from urllib.request import urlretrieve
except ImportError:  # Python 2.x
    from urllib import pathname2url as urlencode
    from urllib import urlretrieve

try: # Python 3.x
    import http.client as httplib 
except ImportError:  # Python 2.x
    import httplib
    

# My function: get ps1_psc score from df
def get_psc_score(df):
    ps_score_l = []
    for i, (ztfid, ra, dec, ps1OID) in enumerate(zip(df.index, df['raMean'], df['decMean'], df['objID'])): 
        
        if i % 1000 == 0: 
            print(f"i={i}")

        if dec >= 0: dec_int = int(math.floor(dec))
        else: dec_int = int(math.ceil(dec))

        # Open fits file corresponding to declination
        with fits.open(f'../ps1_psc/hlsp_ps1-psc_ps1_gpc1_{dec_int}_multi_v1_cat.fits') as F:
            objid_arr=F[1].data['objid']
            ps_score_arr=F[1].data['ps_score']

            try:
                ps_score = ps_score_arr[np.where(objid_arr == ps1OID)[0][0]]
                #print(ps1OID, ps_score)
            except:
                #print(f"{ps1OID} is not in this file. Set ps_score to -99")
                ps_score = -99
            ps_score_l.append(ps_score)
    return ps_score_l
    
    
# My function: get ps1 object id from transient Ra/Dec
def get_ps1_dr2_objid_df_from_ra_dec(ztfid_l, ra_l, dec_l, df_name, radius=1.0/3600.0):
    if len(ztfid_l)>1:
        df_l = []
        for i, (ztfid, ra, dec) in enumerate(zip(ztfid_l, ra_l, dec_l)):
            
            if i % 1000 == 0: 
                print(f"i={i}: Saving!")
                if i != 0:
                    temp_df = pd.concat(df_l)
                    temp_df.to_csv(f'../dataframes/{df_name}_i={i}.csv') 
            
            ztfid = ztfid
            ra, dec = ra, dec
            radius = radius # radius = 1 arcsec
            constraints = {'nDetections.gt':1}

            # strip blanks and weed out blank and commented-out values
            columns = """objID,raMean,decMean,nDetections""".split(',')
            columns = [x.strip() for x in columns]
            columns = [x for x in columns if x and not x.startswith('#')]

            results = ps1cone(ra,dec,radius,release='dr2',columns=columns,**constraints)
            try:
                tab = ascii.read(results)
                tab = tab.to_pandas()
                tab['ztf_object_id'] = ztfid
                tab = tab.set_index('ztf_object_id')

            except: 
                print(f"Bad results for {ztfid}, ra={ra}, dec={dec}. Add dummy var. Continue!")
                print(results)
                
                temp_df = pd.DataFrame(zip([ztfid], [-99], [ra], [dec], [-99]), columns=['ztf_object_id', 'objID', 'raMean', 'decMean', 'nDetections'])
                tab = temp_df.set_index('ztf_object_id')
                
            df_l.append(tab)
        tab_df = pd.concat(df_l)
        
    else: # One entry
        ztfid = ztfid_l
        ra, dec = ra_l, dec_l
        radius = radius # radius = 1 arcsec
        constraints = {'nDetections.gt':1}

        # strip blanks and weed out blank and commented-out values
        columns = """objID,raMean,decMean,nDetections""".split(',')
        columns = [x.strip() for x in columns]
        columns = [x for x in columns if x and not x.startswith('#')]

        results = ps1cone(ra,dec,radius,release='dr2',columns=columns,**constraints)
        try:
            tab = ascii.read(results)
            tab = tab.to_pandas()
            tab['ztf_object_id'] = ztfid
            tab = tab.set_index('ztf_object_id')
        except: 
            print("Bad results. Add dummy var. Return!")
            print(results)
            
            temp_df = pd.DataFrame(zip([ztfid], [-99], [ra], [dec], [-99]), 
            columns=['ztf_object_id', 'objID', 'raMean', 'decMean', 'nDetections'])
            tab = temp_df.set_index('ztf_object_id')

        tab_df = tab
    return tab_df


def ps1cone(ra,dec,radius,table="mean",release="dr1",format="csv",columns=None,
           baseurl="https://catalogs.mast.stsci.edu/api/v0.1/panstarrs", verbose=False,
           **kw):
    """Do a cone search of the PS1 catalog
    
    Parameters
    ----------
    ra (float): (degrees) J2000 Right Ascension
    dec (float): (degrees) J2000 Declination
    radius (float): (degrees) Search radius (<= 0.5 degrees)
    table (string): mean, stack, or detection
    release (string): dr1 or dr2
    format: csv, votable, json
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'nDetections.min':2)
    """
    
    data = kw.copy()
    data['ra'] = ra
    data['dec'] = dec
    data['radius'] = radius
    return ps1search(table=table,release=release,format=format,columns=columns,
                    baseurl=baseurl, verbose=verbose, **data)


def ps1search(table="mean",release="dr1",format="csv",columns=None,
           baseurl="https://catalogs.mast.stsci.edu/api/v0.1/panstarrs", verbose=False,
           **kw):
    """Do a general search of the PS1 catalog (possibly without ra/dec/radius)
    
    Parameters
    ----------
    table (string): mean, stack, or detection
    release (string): dr1 or dr2
    format: csv, votable, json
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'nDetections.min':2).  Note this is required!
    """
    
    data = kw.copy()
    if not data:
        raise ValueError("You must specify some parameters for search")
    checklegal(table,release)
    if format not in ("csv","votable","json"):
        raise ValueError("Bad value for format")
    url = f"{baseurl}/{release}/{table}.{format}"
    if columns:
        # check that column values are legal
        # create a dictionary to speed this up
        dcols = {}
        for col in ps1metadata(table,release)['name']:
            dcols[col.lower()] = 1
        badcols = []
        for col in columns:
            if col.lower().strip() not in dcols:
                badcols.append(col)
        if badcols:
            raise ValueError('Some columns not found in table: {}'.format(', '.join(badcols)))
        # two different ways to specify a list of column values in the API
        # data['columns'] = columns
        data['columns'] = '[{}]'.format(','.join(columns))

# either get or post works
#    r = requests.post(url, data=data)
    r = requests.get(url, params=data)

    if verbose:
        print(r.url)
    r.raise_for_status()
    if format == "json":
        return r.json()
    else:
        return r.text


def checklegal(table,release):
    """Checks if this combination of table and release is acceptable
    
    Raises a VelueError exception if there is problem
    """
    
    releaselist = ("dr1", "dr2")
    if release not in ("dr1","dr2"):
        raise ValueError("Bad value for release (must be one of {})".format(', '.join(releaselist)))
    if release=="dr1":
        tablelist = ("mean", "stack")
    else:
        tablelist = ("mean", "stack", "detection")
    if table not in tablelist:
        raise ValueError("Bad value for table (for {} must be one of {})".format(release, ", ".join(tablelist)))


def ps1metadata(table="mean",release="dr1",
           baseurl="https://catalogs.mast.stsci.edu/api/v0.1/panstarrs"):
    """Return metadata for the specified catalog and table
    
    Parameters
    ----------
    table (string): mean, stack, or detection
    release (string): dr1 or dr2
    baseurl: base URL for the request
    
    Returns an astropy table with columns name, type, description
    """
    
    checklegal(table,release)
    url = f"{baseurl}/{release}/{table}/metadata"
    r = requests.get(url)
    r.raise_for_status()
    v = r.json()
    # convert to astropy table
    tab = Table(rows=[(x['name'],x['type'],x['description']) for x in v],
               names=('name','type','description'))
    return tab


def mastQuery(request):
    """Perform a MAST query.

    Parameters
    ----------
    request (dictionary): The MAST request json object

    Returns head,content where head is the response HTTP headers, and content is the returned data
    """
    
    server='mast.stsci.edu'

    # Grab Python Version 
    version = ".".join(map(str, sys.version_info[:3]))

    # Create Http Header Variables
    headers = {"Content-type": "application/x-www-form-urlencoded",
               "Accept": "text/plain",
               "User-agent":"python-requests/"+version}

    # Encoding the request as a json string
    requestString = json.dumps(request)
    requestString = urlencode(requestString)
    
    # opening the https connection
    conn = httplib.HTTPSConnection(server)

    # Making the query
    conn.request("POST", "/api/v0/invoke", "request="+requestString, headers)

    # Getting the response
    resp = conn.getresponse()
    head = resp.getheaders()
    content = resp.read().decode('utf-8')

    # Close the https connection
    conn.close()

    return head,content


def resolve(name):
    """Get the RA and Dec for an object using the MAST name resolver
    
    Parameters
    ----------
    name (str): Name of object

    Returns RA, Dec tuple with position"""

    resolverRequest = {'service':'Mast.Name.Lookup',
                       'params':{'input':name,
                                 'format':'json'
                                },
                      }
    headers,resolvedObjectString = mastQuery(resolverRequest)
    resolvedObject = json.loads(resolvedObjectString)
    # The resolver returns a variety of information about the resolved object, 
    # however for our purposes all we need are the RA and Dec
    try:
        objRa = resolvedObject['resolvedCoordinate'][0]['ra']
        objDec = resolvedObject['resolvedCoordinate'][0]['decl']
    except IndexError as e:
        raise ValueError("Unknown object '{}'".format(name))
    return (objRa, objDec)

In [2]:
test_df = pd.read_csv('../dataframes/get_ps1_dr2_objid_ghost_hosts_assoc_coords_df.csv')
test_df = test_df.head()
test_df

,ztf_object_id,objID,raMean,decMean,nDetections
0,ZTF17aaaaenu,150620273387282157,27.338759,35.517955,38
1,ZTF17aaaazhi,148200334649204503,33.464886,33.503208,40
2,ZTF17aaaazmz,150890302862623476,30.286253,35.744116,55
3,ZTF17aaaazmz,150890302862863079,30.286215,35.744006,3
4,ZTF17aaabghb,106760772412722499,77.241261,-1.031607,54


In [3]:
test_l = get_psc_score(df=test_df)

i=0


In [4]:
test_l

[0.025833333, 0.9295833, 0.0, -99, 0.00375]

In [5]:
temp_df = pd.DataFrame(zip(['ztftest'], [130581239404351330], [123.9404466], [18.81734214], [-99]), columns=['ztf_object_id', 'objID', 'raMean', 'decMean', 'nDetections'])
tab = temp_df.set_index('ztf_object_id')
tab

,objID,raMean,decMean,nDetections
ztf_object_id,,,,
ztftest,130581239404351330,123.940447,18.817342,-99


In [6]:
get_psc_score(df=tab)

i=0


[-99]

In [7]:
test2_df = pd.read_csv(f'../dataframes/get_ps1_dr2_objid_transient_coords_df_pgal_transient.csv')
test2_df = test2_df[(test2_df.pgal_transient < 0.1) & (test2_df.pgal_transient != -99)]
test2_df = test2_df.set_index('ztf_object_id')
test2_df

,objID,raMean,decMean,nDetections,pgal_transient
ztf_object_id,,,,,
ZTF17aaadvfh,156720101231743602,10.123150,40.602459,52,0.079583
ZTF17aaazdba,135171233206158525,123.320625,22.648309,76,0.078875
ZTF17aadqidc,142391508074825434,150.807460,28.662395,74,0.001250
ZTF18aaacesi,173701275015223476,127.501508,54.752292,57,0.018917
ZTF18aaahyks,167871242161312279,124.216125,49.893012,70,0.071292
...,...,...,...,...,...
ZTF21acjzszh,142731210410615459,121.041057,28.945746,79,0.053179
ZTF21ackmpcb,83073235114471120,323.511427,-20.774255,59,0.000000
ZTF21acldmwy,86950870276547146,87.027691,-17.536090,59,0.010458


In [8]:
dataset_bank_tns_df = pd.read_csv('../dataframes/dataset_bank_90k_tns_df.csv', index_col='ztf_object_id')
dataset_bank_tns_df

,has_tns,tns_cls,spec_z,report_group
ztf_object_id,,,,
ZTF20acpdyyq,No_TNS,None,-99.0,None
ZTF18acfvwwa,No_TNS,None,-99.0,None
ZTF17aaaskvj,No_TNS,None,-99.0,None
ZTF18aczbnhl,No_TNS,None,-99.0,None
ZTF18actbhqt,No_TNS,None,-99.0,None
...,...,...,...,...
ZTF18adairvu,No_TNS,None,-99.0,None
ZTF20aaiwhct,No_TNS,None,-99.0,None
ZTF20aberawv,No_TNS,None,-99.0,None


In [9]:
tns_transient_objid_df = pd.merge(test2_df, dataset_bank_tns_df, left_index=True, right_index=True)
tns_transient_objid_df

,objID,raMean,decMean,nDetections,pgal_transient,has_tns,tns_cls,spec_z,report_group
ztf_object_id,,,,,,,,,
ZTF17aaadvfh,156720101231743602,10.123150,40.602459,52,0.079583,No_TNS,None,-99.000,None
ZTF17aaazdba,135171233206158525,123.320625,22.648309,76,0.078875,Yes_TNS,TDE,0.022,ASAS-SN
ZTF17aadqidc,142391508074825434,150.807460,28.662395,74,0.001250,No_TNS,None,-99.000,None
ZTF18aaacesi,173701275015223476,127.501508,54.752292,57,0.018917,No_TNS,None,-99.000,None
ZTF18aaahyks,167871242161312279,124.216125,49.893012,70,0.071292,No_TNS,None,-99.000,None
...,...,...,...,...,...,...,...,...,...
ZTF21acjzszh,142731210410615459,121.041057,28.945746,79,0.053179,No_TNS,None,-99.000,None
ZTF21ackmpcb,83073235114471120,323.511427,-20.774255,59,0.000000,Yes_TNS,None,-99.000,ALeRCE
ZTF21acldmwy,86950870276547146,87.027691,-17.536090,59,0.010458,Yes_TNS,SN Ia,0.110,ZTF


In [10]:
good_missed_sne_candidates_df = tns_transient_objid_df[tns_transient_objid_df.has_tns == 'No_TNS']
good_missed_sne_candidates_df

,objID,raMean,decMean,nDetections,pgal_transient,has_tns,tns_cls,spec_z,report_group
ztf_object_id,,,,,,,,,
ZTF17aaadvfh,156720101231743602,10.123150,40.602459,52,0.079583,No_TNS,None,-99.0,None
ZTF17aadqidc,142391508074825434,150.807460,28.662395,74,0.001250,No_TNS,None,-99.0,None
ZTF18aaacesi,173701275015223476,127.501508,54.752292,57,0.018917,No_TNS,None,-99.0,None
ZTF18aaahyks,167871242161312279,124.216125,49.893012,70,0.071292,No_TNS,None,-99.0,None
ZTF18aabehyt,142662048071232275,204.807103,28.884739,63,0.004375,No_TNS,None,-99.0,None
...,...,...,...,...,...,...,...,...,...
ZTF21acihfiw,143721418856217687,141.885646,29.772591,50,0.000000,No_TNS,None,-99.0,None
ZTF21acjhvaq,158200187358673932,18.735855,41.836107,33,0.048375,No_TNS,None,-99.0,None
ZTF21acjoxty,149511216343228246,121.634289,34.597995,69,0.000000,No_TNS,None,-99.0,None


In [11]:
# LOTS of AGN/bogus (center of gal)
# But some missed SNe in here
for sn in good_missed_sne_candidates_df.index:
    print(f'https://alerce.online/object/{sn}')

https://alerce.online/object/ZTF17aaadvfh
https://alerce.online/object/ZTF17aadqidc
https://alerce.online/object/ZTF18aaacesi
https://alerce.online/object/ZTF18aaahyks
https://alerce.online/object/ZTF18aabehyt
https://alerce.online/object/ZTF18aacbuof
https://alerce.online/object/ZTF18aacckpk
https://alerce.online/object/ZTF18aacdcas
https://alerce.online/object/ZTF18aaemfad
https://alerce.online/object/ZTF18aaetnmr
https://alerce.online/object/ZTF18aahkypd
https://alerce.online/object/ZTF18aahslvt
https://alerce.online/object/ZTF18aahtzkz
https://alerce.online/object/ZTF18aahvkih
https://alerce.online/object/ZTF18aahwzbd
https://alerce.online/object/ZTF18aahxwwm
https://alerce.online/object/ZTF18aahyuqr
https://alerce.online/object/ZTF18aaibiuc
https://alerce.online/object/ZTF18aaisuxd
https://alerce.online/object/ZTF18aaiuagv
https://alerce.online/object/ZTF18aaiuynw
https://alerce.online/object/ZTF18aaivmkf
https://alerce.online/object/ZTF18aajajad
https://alerce.online/object/ZTF18

https://alerce.online/object/ZTF21aasvfyk
https://alerce.online/object/ZTF21aaswutu
https://alerce.online/object/ZTF21aaswvjt
https://alerce.online/object/ZTF21aatbbjt
https://alerce.online/object/ZTF21aatjajf
https://alerce.online/object/ZTF21aatkipi
https://alerce.online/object/ZTF21aatlkcr
https://alerce.online/object/ZTF21aatwxjf
https://alerce.online/object/ZTF21aauadqq
https://alerce.online/object/ZTF21aaubjch
https://alerce.online/object/ZTF21aaubqdd
https://alerce.online/object/ZTF21aaujdph
https://alerce.online/object/ZTF21aauufso
https://alerce.online/object/ZTF21aauumne
https://alerce.online/object/ZTF21aauunwy
https://alerce.online/object/ZTF21aauuqrt
https://alerce.online/object/ZTF21aauuuxw
https://alerce.online/object/ZTF21aauvbxe
https://alerce.online/object/ZTF21aauwtwz
https://alerce.online/object/ZTF21aauwvhg
https://alerce.online/object/ZTF21aavdmtm
https://alerce.online/object/ZTF21aavdqnk
https://alerce.online/object/ZTF21aavejbv
https://alerce.online/object/ZTF21

In [12]:
# https://alerce.online/object/ZTF17aaadvfh - nova
# https://alerce.online/object/ZTF17aadqidc - AGN/missed transient
# https://alerce.online/object/ZTF18aaiuynw - SN Ia
# https://alerce.online/object/ZTF18aajdimg - AGN/missed transient
# https://alerce.online/object/ZTF18aajkgtr - SN
# https://alerce.online/object/ZTF18aajmlnp - SN
# https://alerce.online/object/ZTF18aakcsqt - SN

# Best candidates for missed will be NO TNS, pgal_transient < 0.1 (strong galaxy preference), pgal_host < 0.1 (strong galaxy preference)
# Will generate a lot of AGN, and other missed SN towards gal center.

# Other good candidates will be be NO TNS, pgal_transient == -99 (no ps1 objid- either missing or transient not at/near gal center), pgal_host < 0.1 (strong galaxy preference)

In [13]:
test2_df2 = pd.read_csv(f'../dataframes/get_ps1_dr2_objid_transient_coords_df_pgal_transient.csv')
test2_df2 = test2_df2[(test2_df2.pgal_transient < 0.1) & (test2_df2.pgal_transient == -99)]
test2_df2 = test2_df2.set_index('ztf_object_id')

tns_transient_objid_df2 = pd.merge(test2_df2, dataset_bank_tns_df, left_index=True, right_index=True)

good_missed_sne_candidates_df2 = tns_transient_objid_df2[tns_transient_objid_df2.has_tns == 'No_TNS']
good_missed_sne_candidates_df2

,objID,raMean,decMean,nDetections,pgal_transient,has_tns,tns_cls,spec_z,report_group
ztf_object_id,,,,,,,,,
ZTF17aaafvwm,103260474717950197,47.471790,-3.949986,3,-99.0,No_TNS,None,-99.0,None
ZTF17aaaiujz,167211141963355377,114.196321,49.345549,93,-99.0,No_TNS,None,-99.0,None
ZTF17aaajtkd,155680241918808622,24.191912,39.739974,69,-99.0,No_TNS,None,-99.0,None
ZTF17aaaslay,147560282198509861,28.219864,32.974382,70,-99.0,No_TNS,None,-99.0,None
ZTF17aaaswgd,114990744991528047,74.499191,5.831305,103,-99.0,No_TNS,None,-99.0,None
...,...,...,...,...,...,...,...,...,...
ZTF21achfboi,142870441552599840,44.155322,29.066039,66,-99.0,No_TNS,None,-99.0,None
ZTF21acogvaq,179681784576665714,178.457763,59.737593,2,-99.0,No_TNS,None,-99.0,None
ZTF21acorwev,199190929403846294,92.940374,75.996275,2,-99.0,No_TNS,None,-99.0,None


In [14]:
# LOTS of stars
# But some missed SNe in here
for sn in good_missed_sne_candidates_df2.index:
    print(f'https://alerce.online/object/{sn}')

https://alerce.online/object/ZTF17aaafvwm
https://alerce.online/object/ZTF17aaaiujz
https://alerce.online/object/ZTF17aaajtkd
https://alerce.online/object/ZTF17aaaslay
https://alerce.online/object/ZTF17aaaswgd
https://alerce.online/object/ZTF17aaaszpi
https://alerce.online/object/ZTF17aaatuld
https://alerce.online/object/ZTF17aaatuld
https://alerce.online/object/ZTF17aaatxql
https://alerce.online/object/ZTF17aaatyyh
https://alerce.online/object/ZTF17aaatyyh
https://alerce.online/object/ZTF17aaaxdcl
https://alerce.online/object/ZTF17aaazoyj
https://alerce.online/object/ZTF17aaazqqd
https://alerce.online/object/ZTF17aaazqqf
https://alerce.online/object/ZTF17aaazrnw
https://alerce.online/object/ZTF17aaazxth
https://alerce.online/object/ZTF17aaazxtr
https://alerce.online/object/ZTF17aaazxtv
https://alerce.online/object/ZTF17aabatmu
https://alerce.online/object/ZTF17aabaujm
https://alerce.online/object/ZTF17aabaynv
https://alerce.online/object/ZTF17aabbksz
https://alerce.online/object/ZTF17

https://alerce.online/object/ZTF19aaedqgq
https://alerce.online/object/ZTF19aaedrwa
https://alerce.online/object/ZTF19aaedufl
https://alerce.online/object/ZTF19aaedwzq
https://alerce.online/object/ZTF19aaemhvy
https://alerce.online/object/ZTF19aaepgpn
https://alerce.online/object/ZTF19aaepirh
https://alerce.online/object/ZTF19aaepkhd
https://alerce.online/object/ZTF19aaepohr
https://alerce.online/object/ZTF19aaeqnae
https://alerce.online/object/ZTF19aaeqois
https://alerce.online/object/ZTF19aaerbms
https://alerce.online/object/ZTF19aaesfep
https://alerce.online/object/ZTF19aaeskgh
https://alerce.online/object/ZTF19aaevarp
https://alerce.online/object/ZTF19aaevqwz
https://alerce.online/object/ZTF19aaevyth
https://alerce.online/object/ZTF19aaewdaf
https://alerce.online/object/ZTF19aaewddd
https://alerce.online/object/ZTF19aaewqpz
https://alerce.online/object/ZTF19aaexirj
https://alerce.online/object/ZTF19aaexkgv
https://alerce.online/object/ZTF19aaeyfmz
https://alerce.online/object/ZTF19

https://alerce.online/object/ZTF21acfbxhh
https://alerce.online/object/ZTF21achfboi
https://alerce.online/object/ZTF21acogvaq
https://alerce.online/object/ZTF21acorwev
https://alerce.online/object/ZTF21acoswbv
https://alerce.online/object/ZTF21acqqnqg


In [15]:
test_df3 = pd.read_csv('../dataframes/get_ps1_dr2_objid_ghost_hosts_assoc_coords_df_pgal_host.csv')
test_df3 = test_df3[(test_df3.pgal_host < 0.1) & (test_df3.pgal_host != -99)]
test_df3 = test_df3.set_index('ztf_object_id')
test_df3

,objID,raMean,decMean,nDetections,pgal_host
ztf_object_id,,,,,
ZTF17aaaaenu,150620273387282157,27.338759,35.517955,38,0.025833
ZTF17aaaazmz,150890302862623476,30.286253,35.744116,55,0.000000
ZTF17aaabghb,106760772412722499,77.241261,-1.031607,54,0.003750
ZTF17aaabuab,105210520766363928,52.076674,-2.322100,68,0.016417
ZTF17aaaccjr,113240693362325305,69.336154,4.370705,74,0.000833
...,...,...,...,...,...
ZTF21acowflo,105441783569413306,178.356941,-2.130975,10,0.010208
ZTF21acqdfea,79683422147160553,342.214736,-23.599836,42,0.024155
ZTF21acqetxn,131013575480825509,357.548079,19.179034,37,0.000000


In [16]:
keep_df = pd.read_csv('../dataframes/keep_df.csv')
keep_df = keep_df.set_index('ztf_object_id')

keep_df

,objID_x,raMean_x,decMean_x,nDetections_x,pgal_host,objID_y,raMean_y,decMean_y,nDetections_y,pgal_transient
ztf_object_id,,,,,,,,,,
ZTF17aaadvfh,156720101231743602,10.123150,40.602459,52,0.079583,156720101231743602,10.123150,40.602459,52,0.079583
ZTF17aaahrni,102970804180328356,80.418019,-4.184979,46,0.004167,102970804232846942,80.423322,-4.186241,4,0.322083
ZTF17aaaiujz,167221141795271735,114.179554,49.350772,24,0.002500,167211141963355377,114.196321,49.345549,93,-99.000000
ZTF17aaajtkd,155680241918808622,24.191912,39.739974,69,-99.000000,155680241918808622,24.191912,39.739974,69,-99.000000
ZTF17aaaslay,147560282198509861,28.219864,32.974382,70,-99.000000,147560282198509861,28.219864,32.974382,70,-99.000000
...,...,...,...,...,...,...,...,...,...,...
ZTF21acogvaq,179681784583745824,178.458176,59.737752,63,0.000000,179681784583745824,178.458176,59.737752,63,0.000000
ZTF21acogvaq,179681784583745824,178.458176,59.737752,63,0.000000,179681784576665714,178.457763,59.737593,2,-99.000000
ZTF21acoswbv,127780486585439993,48.658554,16.491211,40,-99.000000,127780486585439993,48.658554,16.491211,40,-99.000000


In [17]:
keep_df_search_missed_sne_df = pd.merge(keep_df, dataset_bank_tns_df, left_index=True, right_index=True)
keep_df_search_missed_sne_df

,objID_x,raMean_x,decMean_x,nDetections_x,pgal_host,objID_y,raMean_y,decMean_y,nDetections_y,pgal_transient,has_tns,tns_cls,spec_z,report_group
ztf_object_id,,,,,,,,,,,,,,
ZTF17aaadvfh,156720101231743602,10.123150,40.602459,52,0.079583,156720101231743602,10.123150,40.602459,52,0.079583,No_TNS,None,-99.0,None
ZTF17aaahrni,102970804180328356,80.418019,-4.184979,46,0.004167,102970804232846942,80.423322,-4.186241,4,0.322083,Yes_TNS,None,-99.0,GaiaAlerts
ZTF17aaaiujz,167221141795271735,114.179554,49.350772,24,0.002500,167211141963355377,114.196321,49.345549,93,-99.000000,No_TNS,None,-99.0,None
ZTF17aaajtkd,155680241918808622,24.191912,39.739974,69,-99.000000,155680241918808622,24.191912,39.739974,69,-99.000000,No_TNS,None,-99.0,None
ZTF17aaaslay,147560282198509861,28.219864,32.974382,70,-99.000000,147560282198509861,28.219864,32.974382,70,-99.000000,No_TNS,None,-99.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF21acogvaq,179681784583745824,178.458176,59.737752,63,0.000000,179681784583745824,178.458176,59.737752,63,0.000000,No_TNS,None,-99.0,None
ZTF21acogvaq,179681784583745824,178.458176,59.737752,63,0.000000,179681784576665714,178.457763,59.737593,2,-99.000000,No_TNS,None,-99.0,None
ZTF21acoswbv,127780486585439993,48.658554,16.491211,40,-99.000000,127780486585439993,48.658554,16.491211,40,-99.000000,No_TNS,None,-99.0,None


In [18]:
mask = (keep_df_search_missed_sne_df.has_tns == "No_TNS") &\
       (keep_df_search_missed_sne_df.pgal_host < 0.1) &\
        (keep_df_search_missed_sne_df.pgal_host != -99) &\
        (keep_df_search_missed_sne_df.pgal_transient <= 0.1) &\
       (keep_df_search_missed_sne_df.pgal_transient != -99)
keep_df_search_missed_sne_df = keep_df_search_missed_sne_df[mask]
keep_df_search_missed_sne_df

,objID_x,raMean_x,decMean_x,nDetections_x,pgal_host,objID_y,raMean_y,decMean_y,nDetections_y,pgal_transient,has_tns,tns_cls,spec_z,report_group
ztf_object_id,,,,,,,,,,,,,,
ZTF17aaadvfh,156720101231743602,10.123150,40.602459,52,0.079583,156720101231743602,10.123150,40.602459,52,0.079583,No_TNS,None,-99.0,None
ZTF17aadqidc,142391508074825434,150.807460,28.662395,74,0.001250,142391508074825434,150.807460,28.662395,74,0.001250,No_TNS,None,-99.0,None
ZTF18aaacesi,173701274849815785,127.484935,54.754235,22,0.053214,173701275015223476,127.501508,54.752292,57,0.018917,No_TNS,None,-99.0,None
ZTF18aaahyks,167861242216968874,124.221705,49.890171,71,0.011875,167871242161312279,124.216125,49.893012,70,0.071292,No_TNS,None,-99.0,None
ZTF18aacbuof,123201939675568658,193.967509,12.673494,82,0.000000,123201939675568658,193.967509,12.673494,82,0.000000,No_TNS,None,-99.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF21acihfiw,143721418856217687,141.885646,29.772591,50,0.000000,143721418856217687,141.885646,29.772591,50,0.000000,No_TNS,None,-99.0,None
ZTF21acjhvaq,158200187358673932,18.735855,41.836107,33,0.048375,158200187358673932,18.735855,41.836107,33,0.048375,No_TNS,None,-99.0,None
ZTF21acjoxty,149511216343228246,121.634289,34.597995,69,0.000000,149511216343228246,121.634289,34.597995,69,0.000000,No_TNS,None,-99.0,None


In [19]:
# AGN or missed SNe in here
# But no stars!
for sn in keep_df_search_missed_sne_df.index:
    print(f'https://alerce.online/object/{sn}')

https://alerce.online/object/ZTF17aaadvfh
https://alerce.online/object/ZTF17aadqidc
https://alerce.online/object/ZTF18aaacesi
https://alerce.online/object/ZTF18aaahyks
https://alerce.online/object/ZTF18aacbuof
https://alerce.online/object/ZTF18aacckpk
https://alerce.online/object/ZTF18aacdcas
https://alerce.online/object/ZTF18aaemfad
https://alerce.online/object/ZTF18aaetnmr
https://alerce.online/object/ZTF18aahkypd
https://alerce.online/object/ZTF18aahslvt
https://alerce.online/object/ZTF18aahtzkz
https://alerce.online/object/ZTF18aahvkih
https://alerce.online/object/ZTF18aahwzbd
https://alerce.online/object/ZTF18aahxwwm
https://alerce.online/object/ZTF18aahyuqr
https://alerce.online/object/ZTF18aaibiuc
https://alerce.online/object/ZTF18aaisuxd
https://alerce.online/object/ZTF18aaiuagv
https://alerce.online/object/ZTF18aaiuynw
https://alerce.online/object/ZTF18aaivmkf
https://alerce.online/object/ZTF18aajajad
https://alerce.online/object/ZTF18aajarnz
https://alerce.online/object/ZTF18

https://alerce.online/object/ZTF21abengjv
https://alerce.online/object/ZTF21abenlno
https://alerce.online/object/ZTF21abfreru
https://alerce.online/object/ZTF21abftluq
https://alerce.online/object/ZTF21abfvoyj
https://alerce.online/object/ZTF21abfxryd
https://alerce.online/object/ZTF21abgjyjs
https://alerce.online/object/ZTF21abgzkkv
https://alerce.online/object/ZTF21abhrlig
https://alerce.online/object/ZTF21abhsttv
https://alerce.online/object/ZTF21abidgad
https://alerce.online/object/ZTF21abjhfyr
https://alerce.online/object/ZTF21abjqcmb
https://alerce.online/object/ZTF21abjsoxy
https://alerce.online/object/ZTF21abjvqap
https://alerce.online/object/ZTF21abjwduk
https://alerce.online/object/ZTF21abjzrpk
https://alerce.online/object/ZTF21abkqrtc
https://alerce.online/object/ZTF21abmxcyg
https://alerce.online/object/ZTF21abmxfri
https://alerce.online/object/ZTF21abmyknt
https://alerce.online/object/ZTF21abowuoq
https://alerce.online/object/ZTF21abqgvbt
https://alerce.online/object/ZTF21

In [20]:
# problem - (keep_df_search_missed_sne_df.pgal_transient == -99) has SN offset from gal 
# but lets in too many stars with bad host assoc.
# but (keep_df_search_missed_sne_df.pgal_transient != -99) lets is AGN/some SN but cuts out a lot of SN...

In [21]:
#keep_df_search_missed_sne_df.loc['ZTF18abftpsu']

In [22]:
# https://alerce.online/object/ZTF18abnvnqb - missed SN Ia
# https://alerce.online/object/ZTF21aasdcgt - missed SN

In [23]:
dataset_bank_orig = pd.read_csv('../dataframes/dataset_bank_orig.csv')
dataset_bank_orig = dataset_bank_orig.set_index('ztf_object_id')
dataset_bank_orig

,feature_amplitude_magn_r,feature_anderson_darling_normal_magn_r,feature_beyond_1_std_magn_r,feature_beyond_2_std_magn_r,feature_cusum_magn_r,feature_inter_percentile_range_2_magn_r,feature_inter_percentile_range_10_magn_r,feature_inter_percentile_range_25_magn_r,feature_kurtosis_magn_r,feature_linear_fit_slope_magn_r,...,iPSFFlux,iApFlux,iKronFlux,iKronRad,iExtNSigma,gApMag_gKronMag,rApMag_rKronMag,iApMag_iKronMag,7DCD,dist/DLR
ztf_object_id,,,,,,,,,,,,,,,,,,,,,
ZTF20acpdyyq,0.514069,-0.037026,0.230769,0.277350,0.210596,1.028137,0.606000,0.224944,2.176230,-0.001783,...,0.000010,0.000018,0.000016,2.72844,5.283530,-0.122499,-0.208099,-0.147800,66.227214,3.921571
ZTF18acfvwwa,0.593122,-0.303523,0.303030,0.246183,0.272132,1.152622,0.729088,0.356500,0.552720,0.000145,...,0.000024,0.000024,0.000022,2.07345,0.672845,-0.022400,-0.078699,-0.078100,10.879219,4.737505
ZTF20aciscxz,0.285049,-0.425467,0.384615,0.000000,0.375905,0.570099,0.455701,0.273037,-0.871550,0.001117,...,0.000016,0.000016,0.000015,2.09795,-0.243484,-0.117100,-0.058001,-0.122601,31.205722,0.113062
ZTF18abdkosj,1.776150,-0.175299,0.200000,0.000000,0.353906,3.552299,3.552299,1.268549,2.222622,0.005808,...,0.000200,0.000506,0.001007,7.09951,33.325699,0.687300,0.689600,0.746500,15.107083,4.124426
ZTF18abrcoty,0.072900,-0.738972,0.250000,0.000000,0.358274,0.145800,0.145800,0.092799,1.468299,0.004197,...,0.000156,0.000398,0.001222,10.69920,15.298800,1.296299,1.073200,1.217402,20.028593,0.130795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF18aczcgiz,0.542735,-0.273649,0.300000,0.000000,0.372288,1.085470,0.911212,0.659628,-0.834002,-0.001683,...,0.000018,0.000018,0.000016,2.25885,0.992852,0.025900,-0.029100,-0.096601,106.197881,4.446414
ZTF19aaweyrl,0.730350,0.246070,0.372727,0.190693,0.338388,1.277634,0.798832,0.484001,-0.600059,0.000346,...,0.000071,0.000084,0.000083,2.98629,4.623050,-0.060600,-0.051899,-0.008001,50.955001,0.021418
ZTF18aaxheyx,0.932036,-0.808006,0.400000,0.000000,0.372088,1.864073,1.864073,1.287943,-1.375141,-0.002159,...,0.000236,0.000350,0.000438,5.29847,6.611620,0.146198,0.182901,0.246201,32.215306,0.820936


In [24]:
dataset_bank_w_tns_df = pd.merge(dataset_bank_orig, dataset_bank_tns_df, left_index=True, right_index=True)
dataset_bank_w_tns_keep_df = pd.merge(dataset_bank_w_tns_df, keep_df, left_index=True, right_index=True)
#dataset_bank_w_tns_keep_df.to_csv('../dataframes/dataset_bank_w_tns_keep_df.csv')
dataset_bank_w_tns_keep_df

,feature_amplitude_magn_r,feature_anderson_darling_normal_magn_r,feature_beyond_1_std_magn_r,feature_beyond_2_std_magn_r,feature_cusum_magn_r,feature_inter_percentile_range_2_magn_r,feature_inter_percentile_range_10_magn_r,feature_inter_percentile_range_25_magn_r,feature_kurtosis_magn_r,feature_linear_fit_slope_magn_r,...,objID_x,raMean_x,decMean_x,nDetections_x,pgal_host,objID_y,raMean_y,decMean_y,nDetections_y,pgal_transient
ztf_object_id,,,,,,,,,,,,,,,,,,,,,
ZTF17aaahrni,1.501946,-0.235464,0.333333,0.000000,0.300476,3.003893,2.881442,1.473296,-0.318407,0.000616,...,102970804180328356,80.418019,-4.184979,46,0.004167,102970804232846942,80.423322,-4.186241,4,0.322083
ZTF17aaaiujz,0.837050,0.105294,0.125000,0.204124,0.289613,1.674100,0.603275,0.363200,2.376455,-0.000732,...,167221141795271735,114.179554,49.350772,24,0.002500,167211141963355377,114.196321,49.345549,93,-99.000000
ZTF17aaaszpi,0.604250,-0.094789,0.404762,0.000000,0.178456,1.177526,0.919371,0.445400,-0.663844,0.000016,...,107860779037007476,77.903906,-0.110840,30,-99.000000,107850779036737140,77.903698,-0.119396,83,-99.000000
ZTF17aaatuld,0.364731,-0.682132,0.400000,0.000000,0.253155,0.729463,0.729463,0.468043,-1.284994,0.000333,...,152420291435071321,29.143479,37.017250,63,0.013958,152410291469574894,29.147008,37.011882,64,-99.000000
ZTF17aaatuld,0.364731,-0.682132,0.400000,0.000000,0.253155,0.729463,0.729463,0.468043,-1.284994,0.000333,...,152420291435071321,29.143479,37.017250,63,0.013958,152410291470514558,29.146982,37.011638,3,-99.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF21acngnza,0.260479,-0.771551,0.250000,0.000000,0.424403,0.520958,0.520958,0.449773,-2.283457,0.007789,...,158441551680351445,155.168183,42.034115,63,0.000000,158441551680351445,155.168183,42.034115,63,0.000000
ZTF21acogvaq,0.208000,-1.031033,0.500000,0.000000,0.400851,0.416000,0.416000,0.299950,-1.556657,-0.002479,...,179681784576665714,178.457763,59.737593,2,-99.000000,179681784583745824,178.458176,59.737752,63,0.000000
ZTF21acogvaq,0.208000,-1.031033,0.500000,0.000000,0.400851,0.416000,0.416000,0.299950,-1.556657,-0.002479,...,179681784576665714,178.457763,59.737593,2,-99.000000,179681784576665714,178.457763,59.737593,2,-99.000000


In [25]:
cut_stars = (np.log10(dataset_bank_w_tns_keep_df.imomentR1) <=0.1) #cuts some stars. leaves mostly agn, few hostless SN, and stars
#cut_stars = (np.log10(dataset_bank_w_tns_keep_df.imomentR1) >=0.1) # cuts many stars. leaves mostly transients, some AGN and stars
dataset_bank_w_tns_keep_cut_stars_df = dataset_bank_w_tns_keep_df[cut_stars]
dataset_bank_w_tns_keep_cut_stars_df

,feature_amplitude_magn_r,feature_anderson_darling_normal_magn_r,feature_beyond_1_std_magn_r,feature_beyond_2_std_magn_r,feature_cusum_magn_r,feature_inter_percentile_range_2_magn_r,feature_inter_percentile_range_10_magn_r,feature_inter_percentile_range_25_magn_r,feature_kurtosis_magn_r,feature_linear_fit_slope_magn_r,...,objID_x,raMean_x,decMean_x,nDetections_x,pgal_host,objID_y,raMean_y,decMean_y,nDetections_y,pgal_transient
ztf_object_id,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaiujz,0.837050,0.105294,0.125000,0.204124,0.289613,1.674100,0.603275,0.363200,2.376455,-0.000732,...,167221141795271735,114.179554,49.350772,24,0.002500,167211141963355377,114.196321,49.345549,93,-99.000000
ZTF17aaaszpi,0.604250,-0.094789,0.404762,0.000000,0.178456,1.177526,0.919371,0.445400,-0.663844,0.000016,...,107860779037007476,77.903906,-0.110840,30,-99.000000,107850779036737140,77.903698,-0.119396,83,-99.000000
ZTF17aaatyyh,0.855250,-0.024212,0.142857,0.377964,0.300097,1.710500,1.482158,0.441468,2.085195,0.000080,...,144710327674332078,32.767456,30.592905,38,0.381280,144690327642379983,32.764200,30.582815,2,-99.000000
ZTF17aaatyyh,0.855250,-0.024212,0.142857,0.377964,0.300097,1.710500,1.482158,0.441468,2.085195,0.000080,...,144710327674332078,32.767456,30.592905,38,0.381280,144700327639410116,32.763955,30.582942,72,-99.000000
ZTF17aacvxff,0.526488,-0.891846,0.500000,0.000000,0.328482,1.052977,1.052977,0.531337,1.291609,-0.001285,...,91423586287304492,358.628727,-13.813247,29,0.000000,91413586238369676,358.623851,-13.817267,72,-99.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF21acjhvaq,0.095950,-0.252786,0.400000,0.000000,0.437604,0.191900,0.191900,0.185816,-1.913665,0.008230,...,158200187358673932,18.735855,41.836107,33,0.048375,158200187358673932,18.735855,41.836107,33,0.048375
ZTF21acjtjlf,0.152116,-0.352697,0.333333,0.000000,0.309920,0.304232,0.280559,0.067501,1.835493,0.016891,...,91860414875592591,41.487606,-13.448190,55,0.291524,91860414875592591,41.487606,-13.448190,55,0.291524
ZTF21acjyqvb,0.255000,-0.638311,0.222222,0.000000,0.325358,0.510000,0.390883,0.140565,1.017924,-0.002316,...,86210812099383132,81.209947,-18.155898,64,0.404333,86210812099383132,81.209947,-18.155898,64,0.404333


In [26]:
search_miss_mask = (dataset_bank_w_tns_keep_cut_stars_df.has_tns == "No_TNS") &\
       (dataset_bank_w_tns_keep_cut_stars_df.pgal_host < 0.1) &\
        (dataset_bank_w_tns_keep_cut_stars_df.pgal_host != -99) &\
        (dataset_bank_w_tns_keep_cut_stars_df.pgal_transient <= 0.1) #&\
       #(dataset_bank_w_tns_keep_cut_stars_df.pgal_transient == -99)
dataset_bank_w_tns_keep_cut_stars_search_missSNe_df = dataset_bank_w_tns_keep_cut_stars_df[search_miss_mask]
dataset_bank_w_tns_keep_cut_stars_search_missSNe_df

,feature_amplitude_magn_r,feature_anderson_darling_normal_magn_r,feature_beyond_1_std_magn_r,feature_beyond_2_std_magn_r,feature_cusum_magn_r,feature_inter_percentile_range_2_magn_r,feature_inter_percentile_range_10_magn_r,feature_inter_percentile_range_25_magn_r,feature_kurtosis_magn_r,feature_linear_fit_slope_magn_r,...,objID_x,raMean_x,decMean_x,nDetections_x,pgal_host,objID_y,raMean_y,decMean_y,nDetections_y,pgal_transient
ztf_object_id,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaiujz,0.837050,0.105294,0.125000,0.204124,0.289613,1.674100,0.603275,0.363200,2.376455,-0.000732,...,167221141795271735,114.179554,49.350772,24,0.002500,167211141963355377,114.196321,49.345549,93,-99.000000
ZTF17aacvxff,0.526488,-0.891846,0.500000,0.000000,0.328482,1.052977,1.052977,0.531337,1.291609,-0.001285,...,91423586287304492,358.628727,-13.813247,29,0.000000,91413586238369676,358.623851,-13.817267,72,-99.000000
ZTF17aadnacr,0.980750,0.170762,0.238095,0.308607,0.315401,1.961500,1.373483,0.315750,1.269410,-0.001032,...,94690367547192166,36.754666,-11.090204,19,0.004792,-99,36.751740,-11.087389,-99,-99.000000
ZTF18aaacesi,0.409350,-0.324071,0.378378,0.000000,0.331523,0.793213,0.608781,0.381017,-0.922609,0.000629,...,173701274849815785,127.484935,54.754235,22,0.053214,173701275015223476,127.501508,54.752292,57,0.018917
ZTF18aaacwph,1.755684,-0.690928,0.400000,0.000000,0.310152,3.511369,3.511369,1.881042,0.657702,-0.003753,...,180681175655306665,117.565608,60.571528,42,0.046042,180681175851795163,117.585042,60.570188,2,-99.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF21abyflnj,0.182100,-0.378121,0.250000,0.353553,0.252970,0.364201,0.301116,0.100405,1.493708,-0.001878,...,143010268235837784,26.823620,29.181022,57,0.053250,143010268235837784,26.823620,29.181022,57,0.053250
ZTF21acasemn,0.234750,-1.101620,0.500000,0.000000,0.292720,0.469500,0.469500,0.330150,-1.491036,-0.048236,...,171792682915484222,268.291564,53.161335,27,0.007750,-99,268.290667,53.159344,-99,-99.000000
ZTF21acihfiw,0.190450,-0.703335,0.375000,0.000000,0.391861,0.380899,0.341690,0.200100,-0.557860,-0.005700,...,143721418856217687,141.885646,29.772591,50,0.000000,143721418856217687,141.885646,29.772591,50,0.000000


In [60]:
def get_class_name(data):
    class_names = []
    for entry in data:
        if entry['ranking'] == 1 and entry['classifier_version'] in ['stamp_classifier_1.0.0', 'stamp_classifier_1.0.4']:
            class_names.append(entry['class_name'])
        elif entry['ranking'] == 1 and entry['classifier_version'] in ['hierarchical_rf_1.1.0']:
            class_names.append(entry['class_name'])
        else: continue
    unique_class_names = np.unique(class_names)
    if len(unique_class_names) != 0:
        return unique_class_names[0]
    else:
        return None
    
    
# better
def get_class_name2(data):
    # Filter by classifier_version and ranking
    filtered_data = [d for d in data if d['classifier_version'] in ['hierarchical_rf_1.1.0', 'stamp_classifier_1.0.4', 'stamp_classifier_1.0.0'] and d['ranking'] == 1]
    
    # Get unique class names
    unique_class_names = np.unique([d['class_name'] for d in filtered_data])
    
    # Calculate the class_name with the highest probability
    max_prob_class = ''
    max_prob = -1
    for class_name in unique_class_names:
        probs = [d['probability'] for d in filtered_data if d['class_name'] == class_name]
        if len(probs) > 0:
            class_prob = max(probs)
            if class_prob > max_prob:
                max_prob = class_prob
                max_prob_class = class_name
    
    return max_prob_class

# best
def get_class_name3(data):
    # Filter by classifier_version and ranking
    filtered_data = [d for d in data if d['classifier_version'] in ['hierarchical_rf_1.1.0', 'stamp_classifier_1.0.4', 'stamp_classifier_1.0.0'] and d['ranking'] == 1]
    
    # Get unique class names
    unique_class_names = np.unique([d['class_name'] for d in filtered_data])
    
    # Initialize dictionary to store max probabilities for each class_name
    max_prob_dict = {class_name: {'prob': -1, 'version': ''} for class_name in unique_class_names}
    
    # Loop through filtered data and update max_prob_dict
    for d in filtered_data:
        class_name = d['class_name']
        prob = d['probability']
        version = d['classifier_version']
        
        if prob > max_prob_dict[class_name]['prob']:
            max_prob_dict[class_name] = {'prob': prob, 'version': version}
        elif prob == max_prob_dict[class_name]['prob']:
            # Prioritize results from stamp_classifier_1.0.4 before stamp_classifier_1.0.0
            if version == 'hierarchical_rf_1.1.0':
                max_prob_dict[class_name] = {'prob': prob, 'version': version}
            elif version == 'stamp_classifier_1.0.4':
                max_prob_dict[class_name] = {'prob': prob, 'version': version}
            elif version == 'stamp_classifier_1.0.0':
                max_prob_dict[class_name] = {'prob': prob, 'version': version}
            else: print("Not implemented")
    
    # Find class_name with highest probability
    max_prob_class = ''
    max_prob = -1
    for class_name, values in max_prob_dict.items():
        if values['prob'] > max_prob:
            max_prob = values['prob']
            max_prob_class = class_name
    
    return max_prob_class



def get_class_name4(data):
    if data == []: # not classified yet
        top_class_name = "Not_classified"
        return top_class_name
        
    
    # Get unique classifier names and versions
    classifier_names = np.unique([d['classifier_name'] for d in data])
    classifier_versions = np.unique([d['classifier_version'] for d in data])
    
    # Prioritize lc_classifier over stamp_classifier
    if 'lc_classifier' in classifier_names:
        classifier_names = ['lc_classifier', 'stamp_classifier']
    else:
        classifier_names = ['stamp_classifier', 'lc_classifier']
        
    
    for classifier_name in classifier_names:
        for classifier_version in classifier_versions:
            # Filter the data for the current classifier and version
            filtered_data = [d for d in data if d['classifier_name'] == classifier_name 
                                             and d['classifier_version'] == classifier_version]
            print(filtered_data)
            
            # Calculate unique class names and sort by probability in descending order
            unique_class_names = np.unique([d['class_name'] for d in filtered_data])
            unique_class_names = sorted(unique_class_names, key=lambda x: np.max([d['probability'] for d in filtered_data 
                                                                                  if d['class_name'] == x]), reverse=True)
            
            # Get the highest probability for the top class_name
            top_class_name = unique_class_names[0]
            top_class_prob = np.max([d['probability'] for d in filtered_data if d['class_name'] == top_class_name])
            print(top_class_name)
            
            # Check if there is a higher probability for the same class_name in a different classifier version
            for other_version in classifier_versions:
                if other_version == classifier_version:
                    print(other_version)
                    print(classifier_version)
                    continue
                other_filtered_data = [d for d in data if d['classifier_name'] == classifier_name 
                                                         and d['classifier_version'] == other_version
                                                         and d['class_name'] == top_class_name]
                print(other_filtered_data)
                if other_filtered_data:
                    other_class_prob = np.max([d['probability'] for d in other_filtered_data])
                    if other_class_prob > top_class_prob:
                        top_class_prob = other_class_prob
                        
            if not isinstance(top_class_name, str):
                top_class_name == "None"
            
            # Return the top class_name with the highest probability
            return top_class_name

# USE THIS ONE!!!!!!!!!!!! Stratifies like Alerce does
def get_top_class(output):
    top_class = None
    for item in output:
        if item['ranking'] == 1:
            if top_class is None or \
               (item['classifier_name'] == 'lc_classifier' and \
                item['classifier_version'] == 'hierarchical_rf_1.1.0') or \
               (item['classifier_name'] == 'stamp_classifier' and \
                item['classifier_version'] == 'stamp_classifier_1.0.4' and \
                top_class['classifier_version'] != 'hierarchical_rf_1.1.0') or \
               (item['classifier_name'] == 'stamp_classifier' and \
                item['classifier_version'] == 'stamp_classifier_1.0.0' and \
                top_class['classifier_version'] not in ['hierarchical_rf_1.1.0', 'stamp_classifier_1.0.4']):
                top_class = item
    return top_class['class_name'] if top_class is not None else "Not_classified"

In [63]:
prob = alerce.query_probabilities('ZTF18aaxzhzf') # ZTF18aamxjoa also check #"ZTF19aallshi" #"ZTF20abgpnqv"
prob

[{'classifier_name': 'lc_classifier',
  'classifier_version': 'hierarchical_rf_1.1.0',
  'class_name': 'SNIa',
  'probability': 0.1216,
  'ranking': 1},
 {'classifier_name': 'lc_classifier_periodic',
  'classifier_version': 'hierarchical_rf_1.1.0',
  'class_name': 'LPV',
  'probability': 0.184,
  'ranking': 3},
 {'classifier_name': 'lc_classifier_stochastic',
  'classifier_version': 'hierarchical_rf_1.1.0',
  'class_name': 'QSO',
  'probability': 0.136,
  'ranking': 3},
 {'classifier_name': 'lc_classifier_top',
  'classifier_version': 'hierarchical_rf_1.1.0',
  'class_name': 'Transient',
  'probability': 0.38,
  'ranking': 1},
 {'classifier_name': 'lc_classifier_transient',
  'classifier_version': 'hierarchical_rf_1.1.0',
  'class_name': 'SNIa',
  'probability': 0.32,
  'ranking': 1},
 {'classifier_name': 'stamp_classifier',
  'classifier_version': 'stamp_classifier_1.0.0',
  'class_name': 'SN',
  'probability': 0.5038724,
  'ranking': 1},
 {'classifier_name': 'lc_classifier',
  'class

In [64]:
get_top_class(prob)

'SNIa'

In [301]:
dataset_bank_w_tns_keep_cut_stars_search_missSNe_df.head()

,feature_amplitude_magn_r,feature_anderson_darling_normal_magn_r,feature_beyond_1_std_magn_r,feature_beyond_2_std_magn_r,feature_cusum_magn_r,feature_inter_percentile_range_2_magn_r,feature_inter_percentile_range_10_magn_r,feature_inter_percentile_range_25_magn_r,feature_kurtosis_magn_r,feature_linear_fit_slope_magn_r,...,objID_x,raMean_x,decMean_x,nDetections_x,pgal_host,objID_y,raMean_y,decMean_y,nDetections_y,pgal_transient
ztf_object_id,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaiujz,0.837050,0.105294,0.125000,0.204124,0.289613,1.674100,0.603275,0.363200,2.376455,-0.000732,...,167221141795271735,114.179554,49.350772,24,0.002500,167211141963355377,114.196321,49.345549,93,-99.000000
ZTF17aacvxff,0.526488,-0.891846,0.500000,0.000000,0.328482,1.052977,1.052977,0.531337,1.291609,-0.001285,...,91423586287304492,358.628727,-13.813247,29,0.000000,91413586238369676,358.623851,-13.817267,72,-99.000000
ZTF17aadnacr,0.980750,0.170762,0.238095,0.308607,0.315401,1.961500,1.373483,0.315750,1.269410,-0.001032,...,94690367547192166,36.754666,-11.090204,19,0.004792,-99,36.751740,-11.087389,-99,-99.000000
ZTF18aaacesi,0.409350,-0.324071,0.378378,0.000000,0.331523,0.793213,0.608781,0.381017,-0.922609,0.000629,...,173701274849815785,127.484935,54.754235,22,0.053214,173701275015223476,127.501508,54.752292,57,0.018917
ZTF18aaacwph,1.755684,-0.690928,0.400000,0.000000,0.310152,3.511369,3.511369,1.881042,0.657702,-0.003753,...,180681175655306665,117.565608,60.571528,42,0.046042,180681175851795163,117.585042,60.570188,2,-99.000000


In [302]:
%%time
stamp_cls_l = []
for ztfid in dataset_bank_w_tns_keep_cut_stars_search_missSNe_df.index:
    probabilities = alerce.query_probabilities(ztfid)
    stamp_cls = get_class_name4(probabilities)
    stamp_cls_l.append(stamp_cls)
    
dataset_bank_w_tns_keep_cut_stars_search_missSNe_df['stamp_cls'] = stamp_cls_l
dataset_bank_w_tns_keep_cut_stars_search_missSNe_df

CPU times: user 4.25 s, sys: 654 ms, total: 4.9 s
Wall time: 1min


<timed exec>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,feature_amplitude_magn_r,feature_anderson_darling_normal_magn_r,feature_beyond_1_std_magn_r,feature_beyond_2_std_magn_r,feature_cusum_magn_r,feature_inter_percentile_range_2_magn_r,feature_inter_percentile_range_10_magn_r,feature_inter_percentile_range_25_magn_r,feature_kurtosis_magn_r,feature_linear_fit_slope_magn_r,...,raMean_x,decMean_x,nDetections_x,pgal_host,objID_y,raMean_y,decMean_y,nDetections_y,pgal_transient,stamp_cls
ztf_object_id,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaiujz,0.837050,0.105294,0.125000,0.204124,0.289613,1.674100,0.603275,0.363200,2.376455,-0.000732,...,114.179554,49.350772,24,0.002500,167211141963355377,114.196321,49.345549,93,-99.000000,E
ZTF17aacvxff,0.526488,-0.891846,0.500000,0.000000,0.328482,1.052977,1.052977,0.531337,1.291609,-0.001285,...,358.628727,-13.813247,29,0.000000,91413586238369676,358.623851,-13.817267,72,-99.000000,VS
ZTF17aadnacr,0.980750,0.170762,0.238095,0.308607,0.315401,1.961500,1.373483,0.315750,1.269410,-0.001032,...,36.754666,-11.090204,19,0.004792,-99,36.751740,-11.087389,-99,-99.000000,LPV
ZTF18aaacesi,0.409350,-0.324071,0.378378,0.000000,0.331523,0.793213,0.608781,0.381017,-0.922609,0.000629,...,127.484935,54.754235,22,0.053214,173701275015223476,127.501508,54.752292,57,0.018917,AGN
ZTF18aaacwph,1.755684,-0.690928,0.400000,0.000000,0.310152,3.511369,3.511369,1.881042,0.657702,-0.003753,...,117.565608,60.571528,42,0.046042,180681175851795163,117.585042,60.570188,2,-99.000000,VS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF21abyflnj,0.182100,-0.378121,0.250000,0.353553,0.252970,0.364201,0.301116,0.100405,1.493708,-0.001878,...,26.823620,29.181022,57,0.053250,143010268235837784,26.823620,29.181022,57,0.053250,AGN
ZTF21acasemn,0.234750,-1.101620,0.500000,0.000000,0.292720,0.469500,0.469500,0.330150,-1.491036,-0.048236,...,268.291564,53.161335,27,0.007750,-99,268.290667,53.159344,-99,-99.000000,bogus
ZTF21acihfiw,0.190450,-0.703335,0.375000,0.000000,0.391861,0.380899,0.341690,0.200100,-0.557860,-0.005700,...,141.885646,29.772591,50,0.000000,143721418856217687,141.885646,29.772591,50,0.000000,AGN


In [303]:
dataset_bank_w_tns_keep_cut_stars_search_missSNe_df.loc['ZTF20achoozm'].stamp_cls

'Not_classified'

In [304]:
np.unique(dataset_bank_w_tns_keep_cut_stars_search_missSNe_df['stamp_cls'])

array(['AGN', 'Blazar', 'CEP', 'CV/Nova', 'DSCT', 'E', 'LPV',
       'Not_classified', 'Periodic-Other', 'QSO', 'RRL', 'SLSN', 'SN',
       'SNII', 'SNIa', 'SNIbc', 'VS', 'YSO', 'asteroid', 'bogus'],
      dtype=object)

In [305]:
dataset_bank_w_tns_keep_cut_stars_search_missSNe_df_sn_only = dataset_bank_w_tns_keep_cut_stars_search_missSNe_df[(dataset_bank_w_tns_keep_cut_stars_search_missSNe_df.stamp_cls.str.startswith('S'))]
dataset_bank_w_tns_keep_cut_stars_search_missSNe_df_not_sn = dataset_bank_w_tns_keep_cut_stars_search_missSNe_df[~(dataset_bank_w_tns_keep_cut_stars_search_missSNe_df.stamp_cls.str.startswith('S'))]

In [306]:
# Mostly pure SN. Some AGN. Only one star!
for sn, stamp_cls in zip(dataset_bank_w_tns_keep_cut_stars_search_missSNe_df_sn_only.index, dataset_bank_w_tns_keep_cut_stars_search_missSNe_df_sn_only['stamp_cls']):
    print(f'https://alerce.online/object/{sn}', stamp_cls)

https://alerce.online/object/ZTF18aaxzhzf SNIa
https://alerce.online/object/ZTF18aazydub SNIbc
https://alerce.online/object/ZTF18abnwhuf SNIbc
https://alerce.online/object/ZTF18abrlurr SNIa
https://alerce.online/object/ZTF18absljlb SNIbc
https://alerce.online/object/ZTF18absogmu SN
https://alerce.online/object/ZTF18absqohp SNIa
https://alerce.online/object/ZTF18abtgmuw SN
https://alerce.online/object/ZTF18abtmnha SN
https://alerce.online/object/ZTF18abtmnha SN
https://alerce.online/object/ZTF18abuioue SNIa
https://alerce.online/object/ZTF18abulemm SNIa
https://alerce.online/object/ZTF18abvctat SLSN
https://alerce.online/object/ZTF18abvexiz SN
https://alerce.online/object/ZTF18abvkhul SNIa
https://alerce.online/object/ZTF18abvrgjc SNIa
https://alerce.online/object/ZTF18abwhgcb SLSN
https://alerce.online/object/ZTF18abwhsnx SNIbc
https://alerce.online/object/ZTF18abwsfdy SLSN
https://alerce.online/object/ZTF18abxfott SN
https://alerce.online/object/ZTF18acbvgtj SNIa
https://alerce.online

In [307]:
# mostly stars and agn. Few SN as bogus but it would lead to more spurious ones if i allowed them
for sn, stamp_cls in zip(dataset_bank_w_tns_keep_cut_stars_search_missSNe_df_not_sn.index, dataset_bank_w_tns_keep_cut_stars_search_missSNe_df_not_sn['stamp_cls']):
    print(f'https://alerce.online/object/{sn}', stamp_cls)

https://alerce.online/object/ZTF17aaaiujz E
https://alerce.online/object/ZTF17aacvxff VS
https://alerce.online/object/ZTF17aadnacr LPV
https://alerce.online/object/ZTF18aaacesi AGN
https://alerce.online/object/ZTF18aaacwph VS
https://alerce.online/object/ZTF18aaaglfr VS
https://alerce.online/object/ZTF18aabeyws VS
https://alerce.online/object/ZTF18aabfdrc VS
https://alerce.online/object/ZTF18aaccuha AGN
https://alerce.online/object/ZTF18aacpxbn AGN
https://alerce.online/object/ZTF18aaetzue Periodic-Other
https://alerce.online/object/ZTF18aagrmjd E
https://alerce.online/object/ZTF18aagsecq E
https://alerce.online/object/ZTF18aahbell Periodic-Other
https://alerce.online/object/ZTF18aahfaex VS
https://alerce.online/object/ZTF18aahjfau QSO
https://alerce.online/object/ZTF18aahxwct DSCT
https://alerce.online/object/ZTF18aaixdcr VS
https://alerce.online/object/ZTF18aajqbzw RRL
https://alerce.online/object/ZTF18aajqkmc QSO
https://alerce.online/object/ZTF18aajrwaf Periodic-Other
https://alerc

In [166]:
prob = alerce.query_probabilities("ZTF18aalujjg") # ZTF21aaeqism ZTF21aaetqbu ZTF21aacvnsf # ZTF21aaebzrj # ZTF21aagsytt
prob

[{'classifier_name': 'lc_classifier',
  'classifier_version': 'hierarchical_rf_1.1.0',
  'class_name': 'SNIa',
  'probability': 0.00418,
  'ranking': 14},
 {'classifier_name': 'lc_classifier_periodic',
  'classifier_version': 'hierarchical_rf_1.1.0',
  'class_name': 'LPV',
  'probability': 0.014,
  'ranking': 6},
 {'classifier_name': 'lc_classifier_stochastic',
  'classifier_version': 'hierarchical_rf_1.1.0',
  'class_name': 'QSO',
  'probability': 0.4,
  'ranking': 1},
 {'classifier_name': 'lc_classifier_top',
  'classifier_version': 'hierarchical_rf_1.1.0',
  'class_name': 'Transient',
  'probability': 0.022,
  'ranking': 3},
 {'classifier_name': 'lc_classifier_transient',
  'classifier_version': 'hierarchical_rf_1.1.0',
  'class_name': 'SNIa',
  'probability': 0.19,
  'ranking': 3},
 {'classifier_name': 'stamp_classifier',
  'classifier_version': 'stamp_classifier_1.0.0',
  'class_name': 'SN',
  'probability': 0.45442283,
  'ranking': 1},
 {'classifier_name': 'stamp_classifier',
  '

In [112]:
def get_class_name2(data):
    # Filter by classifier_version and ranking
    filtered_data = [d for d in data if d['classifier_version'] in ['stamp_classifier_1.0.4', 'stamp_classifier_1.0.0', 'hierarchical_rf_1.1.0'] and d['ranking'] == 1]
    
    # Get unique class names
    unique_class_names = np.unique([d['class_name'] for d in filtered_data])
    
    # Calculate the class_name with the highest probability
    max_prob_class = ''
    max_prob = -1
    for class_name in unique_class_names:
        probs = [d['probability'] for d in filtered_data if d['class_name'] == class_name]
        if len(probs) > 0:
            class_prob = max(probs)
            if class_prob > max_prob:
                max_prob = class_prob
                max_prob_class = class_name
    
    return max_prob_class

In [167]:
get_class_name3(prob)

'SN'

In [96]:
# Getting detections for an object
probabilities = alerce.query_probabilities("ZTF17aadnacr") # ZTF21aaeqism ZTF21aaetqbu ZTF21aacvnsf # ZTF21aaebzrj # ZTF21aagsytt
probabilities

[{'classifier_name': 'lc_classifier',
  'classifier_version': 'hierarchical_rf_1.1.0',
  'class_name': 'SNIa',
  'probability': 0.001716,
  'ranking': 15},
 {'classifier_name': 'lc_classifier_periodic',
  'classifier_version': 'hierarchical_rf_1.1.0',
  'class_name': 'LPV',
  'probability': 0.412,
  'ranking': 1},
 {'classifier_name': 'lc_classifier_stochastic',
  'classifier_version': 'hierarchical_rf_1.1.0',
  'class_name': 'QSO',
  'probability': 0.058,
  'ranking': 4},
 {'classifier_name': 'lc_classifier_top',
  'classifier_version': 'hierarchical_rf_1.1.0',
  'class_name': 'Transient',
  'probability': 0.022,
  'ranking': 3},
 {'classifier_name': 'lc_classifier_transient',
  'classifier_version': 'hierarchical_rf_1.1.0',
  'class_name': 'SNIa',
  'probability': 0.078,
  'ranking': 4},
 {'classifier_name': 'stamp_classifier',
  'classifier_version': 'stamp_classifier_1.0.0',
  'class_name': 'SN',
  'probability': 0.13038622,
  'ranking': 4},
 {'classifier_name': 'lc_classifier',
  

In [94]:
get_class_name(probabilities)

'CV/Nova'

In [71]:
output = probabilities
#filtered_output = [d for d in output if d['ranking'] == 1 and d['classifier_version'] == 'stamp_classifier_1.0.4']
filtered_output = [d for d in output if d['ranking'] == 1 and d['classifier_version'] != 'hierarchical_rf_1.1.0']


class_names = [d['class_name'] for d in filtered_output]
unique_class_names = np.unique(class_names)

if len(unique_class_names) == 1 and unique_class_names[0] == 'VS':
    print("The class_name is only 'VS'")
else:
    print("The class_name is not only 'VS'")

The class_name is not only 'VS'


In [88]:
def get_class_name(data):
    class_names = []
    for entry in data:
        if entry['ranking'] == 1 and entry['classifier_version'] in ['stamp_classifier_1.0.0', 'stamp_classifier_1.0.4']:
            class_names.append(entry['class_name'])
        elif entry['ranking'] == 1 and entry['classifier_version'] in ['hierarchical_rf_1.1.0']:
            class_names.append(entry['class_name'])
        else: continue
    unique_class_names = np.unique(class_names)
    if len(unique_class_names) != 0:
        return unique_class_names[0]
    else:
        return None

In [86]:
probabilities

[{'classifier_name': 'lc_classifier',
  'classifier_version': 'hierarchical_rf_1.1.0',
  'class_name': 'SNIa',
  'probability': 0.344396,
  'ranking': 2},
 {'classifier_name': 'lc_classifier_periodic',
  'classifier_version': 'hierarchical_rf_1.1.0',
  'class_name': 'LPV',
  'probability': 0.038,
  'ranking': 6},
 {'classifier_name': 'lc_classifier_stochastic',
  'classifier_version': 'hierarchical_rf_1.1.0',
  'class_name': 'QSO',
  'probability': 0.22,
  'ranking': 3},
 {'classifier_name': 'lc_classifier_top',
  'classifier_version': 'hierarchical_rf_1.1.0',
  'class_name': 'Transient',
  'probability': 0.962,
  'ranking': 1},
 {'classifier_name': 'lc_classifier_transient',
  'classifier_version': 'hierarchical_rf_1.1.0',
  'class_name': 'SNIa',
  'probability': 0.358,
  'ranking': 2},
 {'classifier_name': 'lc_classifier',
  'classifier_version': 'hierarchical_rf_1.1.0',
  'class_name': 'SNIbc',
  'probability': 0.407888,
  'ranking': 1},
 {'classifier_name': 'lc_classifier_periodic'

In [87]:
get_class_name(probabilities)

test
test
test
test
test


'CV/Nova'

In [50]:
# select the dictionaries with ranking = 1
rank_1_list = [d for d in probabilities if d['ranking'] == 1]

# get the unique values of class_name in rank_1_list
class_names = np.unique([d['class_name'] for d in rank_1_list])

# check if class_names only contains 'VS'
if len(class_names) == 1 and class_names[0] == 'VS':
    print("class_name is only 'VS'")
else:
    print("class_name is not only 'VS'")

class_name is not only 'VS'


In [ ]:
from astro_ghost.starSeparation import * 
df_gals, df_stars = separateStars_STRM(df)



In [36]:
# did this get rid of stars? NOt perfectly, but better!... basically AGN and SN. Few stars. Includes hostless SN :) )
for sn in dataset_bank_w_tns_keep_cut_stars_search_missSNe_df.index:
    print(f'https://alerce.online/object/{sn}')

https://alerce.online/object/ZTF17aaaiujz
https://alerce.online/object/ZTF17aacvxff
https://alerce.online/object/ZTF17aadnacr
https://alerce.online/object/ZTF18aaacwph
https://alerce.online/object/ZTF18aaaglfr
https://alerce.online/object/ZTF18aabeyws
https://alerce.online/object/ZTF18aabfdrc
https://alerce.online/object/ZTF18aaccuha
https://alerce.online/object/ZTF18aacpxbn
https://alerce.online/object/ZTF18aaetzue
https://alerce.online/object/ZTF18aagrmjd
https://alerce.online/object/ZTF18aagsecq
https://alerce.online/object/ZTF18aahbell
https://alerce.online/object/ZTF18aahfaex
https://alerce.online/object/ZTF18aahjfau
https://alerce.online/object/ZTF18aahxwct
https://alerce.online/object/ZTF18aaixdcr
https://alerce.online/object/ZTF18aajqbzw
https://alerce.online/object/ZTF18aajrwaf
https://alerce.online/object/ZTF18aalsfbd
https://alerce.online/object/ZTF18aalujjg
https://alerce.online/object/ZTF18aamsdqe
https://alerce.online/object/ZTF18aaojnbl
https://alerce.online/object/ZTF18

In [ ]:
# https://alerce.online/object/ZTF21abxyydw - Missed SN
# https://alerce.online/object/ZTF21abxoxhb - missed SN Ia
# https://alerce.online/object/ZTF21abxoxhb - missed SN Ia
# https://alerce.online/object/ZTF21abwulxy - missed SN CC
# https://alerce.online/object/ZTF21abvatnb - missed SN Ia
# https://alerce.online/object/ZTF21aaualdi - missed SLSN
# Missed 

In [31]:
dataset_bank_w_tns_keep_cut_stars_search_missSNe_df.loc['ZTF21aaosrri'].imomentR1

5.079269886016846

In [38]:
np.log10(dataset_bank_w_tns_keep_cut_stars_search_missSNe_df.loc['ZTF21aagsytt'].imomentR1)

-0.002511820780592096

In [ ]:
dataset_bank_w_tns_keep_cut_stars_search_missSNe_df.loc['ZTF18aazydub'].has_tns